In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_json('intents.json')

In [3]:
df.shape

(14, 1)

In [4]:
df.columns


Index(['intents'], dtype='object')

In [5]:
df.describe()

,intents
count,14
unique,14
top,"{'tag': 'hospital_search', 'patterns': ['Looku..."
freq,1


In [6]:
df.head(10)

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi there', '..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'noanswer', 'patterns': [], 'responses..."
4,"{'tag': 'options', 'patterns': ['How you could..."
5,"{'tag': 'adverse_drug', 'patterns': ['How to c..."
6,"{'tag': 'blood_pressure', 'patterns': ['Open b..."
7,"{'tag': 'blood_pressure_search', 'patterns': [..."
8,"{'tag': 'search_blood_pressure_by_patient_id',..."
9,"{'tag': 'pharmacy_search', 'patterns': ['Find ..."


In [29]:
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes= []
documents = []
ignore_words = ['?', ',','!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [22]:
print(ignore_words)

['?', ',', '!']


In [23]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [24]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [25]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [26]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
print(words)
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

['hi', 'there', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodbye', 'nice', 'chatting', 'to', 'you', 'bye', 'till', 'next', 'time', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'awesome', 'thanks', 'thanks', 'for', 'helping', 'me', 'how', 'you', 'could', 'help', 'me', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provide', 'how', 'you', 'can', 'be', 'helpful', 'what', 'support', 'is', 'offered', 'how', 'to', 'check', 'adverse', 'drug', 'reaction', 'open', 'adverse', 'drug', 'module', 'give', 'me', 'a', 'list', 'of', 'drug', 'causing', 'adverse', 'behavior', 'list', 'all', 'drug', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'which', 'drug', 'dont', 'have', 'adverse', 'reaction', 'open', 'blood', 'pressure', 'module', 'task', 'related', 'to', 'blood', 'pressure', 'blood', 'pressure', 'data', 'entry', 'i', 'want', 'to', 'log', 'blood', 'pressure', 'result', 'blood', 'pressure', 'data', '

In [27]:
print(words)

["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [28]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/home/mav/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [30]:
#build the model
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")




Epoch 1/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2719 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1105 - accuracy: 0.2128
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0175 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0015 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8796 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9016 - accuracy: 0.3191
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6629 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4538 - accuracy: 0.6596
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4007 - accuracy: 0.6596
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 1.1013 - accuracy: 0.7234
Epoch 11/

10/10 [==============================] - 0s 4ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 4ms/step - loss: 0.1185 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 7ms/step - loss: 0.1280 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0967 - accuracy: 0.9574
Epoch 89/200
10/10 [==============================] - 0s 5ms/step - loss: 0.1668 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0162 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0491 - accuracy: 0.9787
Epoch 92/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0347 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 4ms/step - loss: 0.0474 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0494 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0461 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0190 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0192 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0130 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0186 - accuracy: 1.0000
Epoc